In [3]:
import joblib
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve
from functools import reduce
np.set_printoptions(suppress=True)
newfeature = pd.read_csv('feature_selected.csv')
dataset=pd.read_csv('multi_class_label.csv')
data_train = np.array(newfeature.iloc[:1040, 1:])
label_train=np.array(dataset.iloc[:1040,2])
data_test = np.array(newfeature.iloc[1040:, 1:])
label_test=np.array(dataset.iloc[1040:,2])
# print(data_train.shape)
# print(data_test.shape)
# print(label_train.shape)
# print(label_test.shape)
# print(label_test.shape)

ET = joblib.load('ET_all.pkl')
RF = joblib.load('RF_all.pkl')
GBDT = joblib.load('GBDT_all.pkl')
Train1 = ET.predict_proba(data_train)[:, 1]
Train2 = RF.predict_proba(data_train)[:, 1]
Train3 = GBDT.predict_proba(data_train)[:, 1]

trainstack = np.vstack([[Train1],
                       [Train2],
                       [Train3]]).T

Test1 = ET.predict_proba(data_test)[:, 1]
Test2 = RF.predict_proba(data_test)[:, 1]
Test3 = GBDT.predict_proba(data_test)[:, 1]
teststack = np.vstack([[Test1],
                       [Test2],
                       [Test3]]).T
cl_final = joblib.load('stack_all.pkl')
score = cl_final.score(teststack, label_test)
y_pred = cl_final.predict(teststack)

# 评估模型性能
acc = accuracy_score(label_test, y_pred)
tn, fp, fn, tp = confusion_matrix(label_test, y_pred).ravel()
sn = tp / (tp + fn)
sp = tn / (tn + fp)
mcc = ((tp*tn) - (fp*fn)) / (((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5)
auc = roc_auc_score(label_test, cl_final.predict_proba(teststack)[:, 1])

# 将评估结果写入csv文件
result = pd.DataFrame({
    'accuracy': [acc],
    'sensitivity': [sn],
    'specificity': [sp],
    'MCC': [mcc],
    'AUC': [auc]
})
result

,accuracy,sensitivity,specificity,MCC,AUC
0,0.923664,0.923664,0.923664,0.847328,0.968155
